### Import Libraries

In [1]:
try:
    from elasticsearch import Elasticsearch,helpers
    import os
    import sys
    import elasticsearch
    import pandas as pd
except Exception as e:
    print("The following error occured:",e)

### Connect to Elasticsearch

In [13]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [14]:
#Lets connect to the server, we are using the python client to do so

def connect_database():
    if es.ping():
        print("connected")
    else:
        print("Failed")
    return es
connect_database()

connected


<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [15]:
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [25]:
#### Check files in directory

In [3]:
for filename in os.listdir():
    print(filename)

.git
.ipynb_checkpoints
combined_stock_data.csv
data
ElasticSearchbasics.ipynb
Elk_google_loc.ipynb
nlpextractor.ipynb
stock_project.ipynb
Untitled.ipynb


### Create an index

In [54]:
#before creating you might want to delete the index so that you don't  overload the cluster
es.indices.delete(index='gtest1', ignore=[400, 404])

{'acknowledged': True}

In [55]:
es.indices.create(index='gtest1', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'gtest1'}

### Import File

In [20]:
df = pd.read_json(r"C:\Users\parth\2022\notebooks\data\Takeout\Location History\Semantic Location History\2021\2021_DECEMBER.json")
# print(df.isnull().sum())

### Combine all 2021 data

In [68]:
os.listdir(r".\data\Takeout\Location History\Semantic Location History\2021")

['2021_APRIL.json',
 '2021_AUGUST.json',
 '2021_DECEMBER.json',
 '2021_FEBRUARY.json',
 '2021_JANUARY.json',
 '2021_JULY.json',
 '2021_JUNE.json',
 '2021_MARCH.json',
 '2021_MAY.json',
 '2021_NOVEMBER.json',
 '2021_OCTOBER.json',
 '2021_SEPTEMBER.json']

In [72]:
data_21 = []
for year in os.listdir("./data/Takeout/Location History/Semantic Location History/"):
    for file in os.listdir("./data/Takeout/Location History/Semantic Location History/"+year):
        df = pd.read_json("./data/Takeout/Location History/Semantic Location History/"+year+"/"+file)
        df2 = df.to_dict('records')
        for i in range(len(df2)):
            line = df2[i]

            if(line.get("timelineObjects").get("placeVisit")!=None):

                doc = {
                'location_latitude':line.get("timelineObjects").get("placeVisit").get("location").get("latitudeE7")/10000000,
                'location_longitude':line.get("timelineObjects").get("placeVisit").get("location").get("longitudeE7")/10000000,
                'address':line.get("timelineObjects").get("placeVisit").get("location").get("address"),
                'location_name':line.get("timelineObjects").get("placeVisit").get("location").get("name"),
                'start_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("startTimestamp"),
                'end_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("endTimestamp")}
                data_21.append(doc)

In [73]:
df = pd.DataFrame.from_dict(data_21)

,location_latitude,location_longitude,address,location_name,start_timestamp,end_timestamp
0,26.915530,75.809993,"C-16, Bhagwan Das Road, Panch Batti, Jaipur, R...",Rajmandir Cinema,2016-12-25T13:58:09.686Z,2016-12-25T14:49:09.355Z
1,26.911086,75.795220,"A-5, Ground Floor, Inside Suryavanshi Pearl, S...",Natural Ice Cream,2016-12-25T15:20:12.264Z,2016-12-25T15:27:26.513Z
2,28.600155,77.226959,"46, Khan Market Rd, Rabindra Nagar, New Delhi,...",Khan Market,2016-12-27T13:10:21Z,2016-12-27T15:08:13.660Z
3,28.505675,77.217034,"Sainik Farms Dispensary, Sainik Farms, Khanpur...",Eklavaya Group|Sainik Farms Central Avenue,2016-12-27T15:55:23Z,2016-12-28T04:51:53.617Z
4,28.539542,77.143911,"Sector - C, Pocket - 9, Vasant Kunj, New Delhi...",Community Park,2016-12-28T05:25:37Z,2016-12-28T07:24:51.652Z
...,...,...,...,...,...,...
1364,40.638500,-74.019466,"430 63rd St\nBrooklyn, NY 11220\nUSA",430 63rd St,2022-01-17T10:18:46.517Z,2022-01-17T10:25:39.493Z
1365,40.627074,-74.024361,"469 79th St\nBrooklyn, NY 11209\nUSA",469 79th St,2022-01-17T10:33:52.563Z,2022-01-18T16:29:05.335Z
1366,40.633588,-74.020598,"6817 5th Ave\nBrooklyn, NY 11220\nUSA",Alpine Cinema,2022-01-18T18:44:42.784Z,2022-01-18T20:53:50.871Z
1367,40.632364,-74.027143,"7315 3rd Ave\nBrooklyn, NY 11209\nUSA",Taj Mahal,2022-01-18T21:02:32.754Z,2022-01-18T22:16:49.747Z


In [7]:
df[:8]

,timelineObjects
0,{'activitySegment': {'startLocation': {'latitu...
1,{'placeVisit': {'location': {'latitudeE7': 406...
2,{'activitySegment': {'startLocation': {'latitu...
3,{'placeVisit': {'location': {'latitudeE7': 406...
4,{'placeVisit': {'location': {'latitudeE7': 406...
5,{'placeVisit': {'location': {'latitudeE7': 406...
6,{'placeVisit': {'location': {'latitudeE7': 406...
7,{'activitySegment': {'startLocation': {'latitu...


In [21]:
df2 = df.to_dict('records')

In [22]:
# add ids to the documents
for i in range(len(df2)):
    df2[i]["id"] = i

In [23]:
df2[0]

{'timelineObjects': {'activitySegment': {'startLocation': {'latitudeE7': 406270797,
    'longitudeE7': -740243470,
    'sourceInfo': {'deviceTag': 657537}},
   'endLocation': {'latitudeE7': 406254813,
    'longitudeE7': -740240442,
    'sourceInfo': {'deviceTag': 657537}},
   'duration': {'startTimestamp': '2021-12-01T19:55:01.136Z',
    'endTimestamp': '2021-12-01T20:04:06.189Z'},
   'distance': 201,
   'activityType': 'WALKING',
   'confidence': 'HIGH',
   'activities': [{'activityType': 'WALKING',
     'probability': 90.53097367286682},
    {'activityType': 'IN_PASSENGER_VEHICLE', 'probability': 4.127069190144539},
    {'activityType': 'CYCLING', 'probability': 2.461812272667885},
    {'activityType': 'IN_BUS', 'probability': 0.8563749492168421},
    {'activityType': 'STILL', 'probability': 0.828239694237709},
    {'activityType': 'RUNNING', 'probability': 0.47490238212049},
    {'activityType': 'IN_SUBWAY', 'probability': 0.22579405922442602},
    {'activityType': 'IN_TRAM', 'proba

In [7]:
# check what the data looks like now
for c,line in enumerate(df2):
    print(c,line)
    break

0 {'timelineObjects': {'activitySegment': {'startLocation': {'latitudeE7': 406270797, 'longitudeE7': -740243470, 'sourceInfo': {'deviceTag': 657537}}, 'endLocation': {'latitudeE7': 406254813, 'longitudeE7': -740240442, 'sourceInfo': {'deviceTag': 657537}}, 'duration': {'startTimestamp': '2021-12-01T19:55:01.136Z', 'endTimestamp': '2021-12-01T20:04:06.189Z'}, 'distance': 201, 'activityType': 'WALKING', 'confidence': 'HIGH', 'activities': [{'activityType': 'WALKING', 'probability': 90.53097367286682}, {'activityType': 'IN_PASSENGER_VEHICLE', 'probability': 4.127069190144539}, {'activityType': 'CYCLING', 'probability': 2.461812272667885}, {'activityType': 'IN_BUS', 'probability': 0.8563749492168421}, {'activityType': 'STILL', 'probability': 0.828239694237709}, {'activityType': 'RUNNING', 'probability': 0.47490238212049}, {'activityType': 'IN_SUBWAY', 'probability': 0.22579405922442602}, {'activityType': 'IN_TRAM', 'probability': 0.18923425814136802}, {'activityType': 'MOTORCYCLING', 'prob

In [26]:
line = df2[0]

In [27]:
line.get("timelineObjects")

{'activitySegment': {'startLocation': {'latitudeE7': 406270797,
   'longitudeE7': -740243470,
   'sourceInfo': {'deviceTag': 657537}},
  'endLocation': {'latitudeE7': 406254813,
   'longitudeE7': -740240442,
   'sourceInfo': {'deviceTag': 657537}},
  'duration': {'startTimestamp': '2021-12-01T19:55:01.136Z',
   'endTimestamp': '2021-12-01T20:04:06.189Z'},
  'distance': 201,
  'activityType': 'WALKING',
  'confidence': 'HIGH',
  'activities': [{'activityType': 'WALKING', 'probability': 90.53097367286682},
   {'activityType': 'IN_PASSENGER_VEHICLE', 'probability': 4.127069190144539},
   {'activityType': 'CYCLING', 'probability': 2.461812272667885},
   {'activityType': 'IN_BUS', 'probability': 0.8563749492168421},
   {'activityType': 'STILL', 'probability': 0.828239694237709},
   {'activityType': 'RUNNING', 'probability': 0.47490238212049},
   {'activityType': 'IN_SUBWAY', 'probability': 0.22579405922442602},
   {'activityType': 'IN_TRAM', 'probability': 0.18923425814136802},
   {'activit

In [62]:
data = []
for i in range(len(df2)):
    line = df2[i]

    if(line.get("timelineObjects").get("placeVisit")!=None):
       
        doc = {
        'location_latitude':line.get("timelineObjects").get("placeVisit").get("location").get("latitudeE7")/10000000,
        'location_longitude':line.get("timelineObjects").get("placeVisit").get("location").get("longitudeE7")/10000000,
        'address':line.get("timelineObjects").get("placeVisit").get("location").get("address"),
        'location_name':line.get("timelineObjects").get("placeVisit").get("location").get("name"),
        'start_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("startTimestamp"),
        'end_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("endTimestamp")}
        data.append(doc)


### Convert dict to dictionary

In [63]:
pd.DataFrame.from_dict(data)

,location_latitude,location_longitude,address,location_name,start_timestamp,end_timestamp
0,40.625425,-74.024044,"8007 5th Ave\nBrooklyn, NY 11209\nUSA",Farmacon Pharmacy,2021-12-01T20:04:06.189Z,2021-12-01T20:10:13.193Z
1,40.627074,-74.024361,"469 79th St\nBrooklyn, NY 11209\nUSA",469 79th St,2021-12-01T20:16:17.219Z,2021-12-01T21:28:33.070Z
2,40.627074,-74.024361,"469 79th St\nBrooklyn, NY 11209\nUSA",469 79th St,2021-12-18T18:21:40.404Z,2021-12-18T19:13:55.515Z
3,40.627074,-74.024361,"469 79th St\nBrooklyn, NY 11209\nUSA",469 79th St,2021-12-20T04:25:35.434Z,2021-12-20T06:54:12.894Z
4,40.638500,-74.019466,"430 63rd St\nBrooklyn, NY 11220\nUSA",430 63rd St,2021-12-21T08:54:18.946Z,2021-12-21T09:22:37Z
5,40.627074,-74.024361,"469 79th St\nBrooklyn, NY 11209\nUSA",469 79th St,2021-12-21T09:26:37Z,2021-12-21T18:56:49.505Z
6,40.727565,-73.993012,"383 Lafayette St\nNew York, NY 10003\nUSA",StudentLink Center,2021-12-21T19:54:50Z,2021-12-22T18:11:49.533Z
7,40.684861,-73.994385,"265 Court St\nBrooklyn, NY 11231\nUSA",Cobble Hill Cinemas,2021-12-22T18:30:26.307Z,2021-12-22T21:11:30.720Z
8,40.692251,-73.987279,"Brooklyn, NY 11201\nUSA",Jay St - MetroTech,2021-12-22T21:31:39.073Z,2021-12-22T21:38:41.782Z
9,40.731114,-74.001223,"340 6th Ave\nNew York, NY 10011\nUSA",W 4 St - Wash Sq,2021-12-22T21:50:21.288Z,2021-12-22T21:58:42.930Z


In [59]:
es.index(index = "gtest1",id = 1,body = doc)

C:\Users\parth\AppData\Local\Temp\ipykernel_712\609001746.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index = "gtest1",id = 1,body = doc)


RequestError: RequestError(400, 'mapper_parsing_exception', 'geo_point expected')

In [58]:
es.indices.get_mapping(index = 'gtest1',doc_type="document",include_type_name=True)

{'gtest1': {'mappings': {'document': {'properties': {'address': {'type': 'text',
      'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
     'end_timestamp': {'type': 'date'},
     'location_latitude': {'type': 'geo_point'},
     'location_longitude': {'type': 'geo_point'},
     'location_name': {'type': 'text',
      'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
     'start_timestamp': {'type': 'date'}}}}}}

#### Important Note

We see that es is not taking longitude and latitude as geopoint locations, one way to crrect this is to define mappings ourselves
<li> Delete the index </li>
<li> define mappings using <a>put_mapping method</a></li>

In [56]:
es.indices.put_mapping(
    index = "gtest1",
    doc_type = "document",
    include_type_name=True,
    body = {'properties': 
            {'address': 
             {'type': 'text',
              'fields': 
                      {'keyword': 
                         {'type': 'keyword', 'ignore_above': 256}}},
             'end_timestamp': {'type': 'date'},
             'location_latitude': {'type': 'geo_point'},
             'location_longitude': {'type': 'geo_point'},
             'location_name': {'type': 'text',
              'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
             'start_timestamp': {'type': 'date'}}})

{'acknowledged': True}

In [20]:
def generator(df2,database_name,table_name,id_column):
    for c,line in enumerate(df2):
        if(line.get("timelineObjects").get("placeVisit")!=None):
            print(c)

            yield{
                '_index' : database_name,
                '_type' : table_name,
                '_id': line.get(id_column),
                '_source': {

                        'location_latitude':line.get("timelineObjects").get("placeVisit").get("location").get("latitudeE7"),
                        'location_longitude':line.get("timelineObjects").get("placeVisit").get("location").get("longitudeE7"),
                        'address':line.get("timelineObjects").get("placeVisit").get("location").get("address"),
                        'location_name':line.get("timelineObjects").get("placeVisit").get("location").get("name"),
                        'start_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("startTimestamp"),
                        'end_timestamp':line.get("timelineObjects").get("placeVisit").get("duration").get("endTimestamp")
                        # 'distance_meters':line.get("timelineObjects",['No Data']).get("activitySegment",['No Data']).get("waypointPath",['No Data']).get("distanceMeters",['No Data']),
                        # 'travel_mode':line.get("timelineObjects",['No Data']).get("activitySegment",['No Data']).get("waypointPath",['No Data']).get("travelMode",['No Data'])                                                                                                
                    # get returns the value of a key

                }
            }
    # raise StopIteration
    

In [21]:
gen = generator(df2,"g_test","glocation","id")

In [22]:
next(gen)

1


{'_index': 'g_test',
 '_type': 'glocation',
 '_id': 1,
 '_source': {'location_latitude': 406254253,
  'location_longitude': -740240437,
  'address': '8007 5th Ave\nBrooklyn, NY 11209\nUSA',
  'location_name': 'Farmacon Pharmacy',
  'start_timestamp': '2021-12-01T20:04:06.189Z',
  'end_timestamp': '2021-12-01T20:10:13.193Z'}}

In [23]:
es.indices.create(index="g_test2",ignore = 400)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [g_test2/aO_nIgMrRzO8mx09rkaqfQ] already exists',
    'index_uuid': 'aO_nIgMrRzO8mx09rkaqfQ',
    'index': 'g_test2'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [g_test2/aO_nIgMrRzO8mx09rkaqfQ] already exists',
  'index_uuid': 'aO_nIgMrRzO8mx09rkaqfQ',
  'index': 'g_test2'},
 'status': 400}

In [24]:
try:
    res = helpers.bulk(es,generator(df2,"g_test2","glocation","id"))
    print("Working")
except Exception as e:
    print("done",e)


1
3
4
5
6
8
11
13
15
17
19
23
25
27
29
31
33
35
37
39
41
44
48
50
52
55
57
59
61
63
65
67
70
72
76
78
80
82
84
86
88
90
93
95
97
99
101
103
106
108
Working


C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [13]:
# Database setup

In [14]:
es.indices.create(index = "mydatabasename",ignore = 400)

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mydatabasename'}

In [123]:
res = es.indices.get_alias("*")
for name in res:
    print(name)

.kibana-event-log-7.16.3-000003
g_test
icecream_kaggle
.async-search
.kibana_7.16.3_001
greencan_demo
.kibana-event-log-7.16.3-000001
.kibana_task_manager_7.16.3_001
.apm-custom-link
.tasks
.apm-agent-configuration
stocks2020to2021
netflix_sample
.kibana-event-log-7.16.3-000002
person


C:\Users\parth\AppData\Local\Temp\ipykernel_31560\516184771.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.get_alias("*")
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: this request accesses system indices: [.async-search, .kibana_7.16.3_001, .kibana_task_manager_7.16.3_001, .apm-custom-link, .tasks, .apm-agent-configuration], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


## Inserting data into kibana

In [19]:
record1 = {
    "name" : "Parth",
    "age" :26,
    
    
}
record2 = {
    "name" : "Samarth",
    "age" :20,
    
    
}

In [20]:
es.indices.create(index="person",ignore = 400)

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'person'}

In [17]:
resp = es.index(index = "g_test",doc_type='glocation',body = df2[0])

C:\Users\parth\AppData\Local\Temp\ipykernel_5688\2801456637.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(index = "g_test",doc_type='glocation',body = df2[0])
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


In [19]:
for i in range(len(df)):
    es.index(index = "g_test2",doc_type='glocation',body = df2[i])
    

C:\Users\parth\AppData\Local\Temp\ipykernel_5688\2628710241.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index = "g_test2",doc_type='glocation',body = df2[i])


In [18]:
resp

{'_index': 'g_test',
 '_type': 'glocation',
 '_id': '8gPnbIAB-WKrEUGjvlT_',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 2}

In [141]:
query = {
    "query" : {
        "match_all":{}
    }
}
    
res = es.search(index='g_test',body = query   )

C:\Users\parth\AppData\Local\Temp\ipykernel_31560\1110477524.py:7: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index='g_test',body = query   )
